In [1]:
import time
import os
import shutil
import sys
import h5py
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
import random                                 as rnd
import math

LocalFlg         = 'False'
if (not LocalFlg):
    WORKSPACE_PATH = "/home/venturi/WORKSPACE/"
    IP             = '128.174.245.172'
    User           = "venturi"
else:
    WORKSPACE_PATH = os.getenv('WORKSPACE_PATH')  
    
import matplotlib as mpl
import matplotlib.pyplot                      as plt
plt.style.use(WORKSPACE_PATH+'/SurQCT/surqct/scripts/postprocessing/presentation.mplstyle')

print("="*50)
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [2]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

Dimension = 'nondim'
RatesType           = 'KExcit'
# ExcitType           = 'KInel'
RatesTypeHDF5       = 'Exch_1'
# NNRunIdx            = 9
ExcitType           = 'KExch'
NNRunIdx            = 6

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures_nondim/' + ExcitType + '/' 

TTransVec           = [1500.0,2500.0,5000.0,6000.0,8000.0,10000.0,12000.0,15000.0,17500.0,20000.0]

MinRate             = 1.e-18

In [3]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata, read_sampledinitiallevels

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...
/home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/


In [4]:
#===================================================================================================================================
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/'
InputData.FldrtoHDF5NN    = WORKSPACE_PATH  + '/Air_Database/HDF5_Database_NN/'
InputData.PathtoHDF5NN    = WORKSPACE_PATH  + '/Air_Database/HDF5_Database_NN/O3_UMN.hdf5'

InputData.DefineModelIntFlg  = 0
InputData.TrainIntFlg        = 0


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [5]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model

NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['log_EVib_i', 'log_ERot_i', 'ri_i', 'log_rorMin_i', 'TTran_i', 'log_EVib_Delta', 'log_ERot_Delta', 'ri_Delta', 'log_rorMin_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KExch//Run_6/Params//ModelCheckpoint/cp-0831.ckpt


In [6]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
MinValueTrain      = 1.e-16 * InputData.MultFact
MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

[SurQCT]:   Reading Variables:  ['ri', 'vqn', 'log_rorMin', 'log_ERot', 'jqn', 'log_EVib']


In [7]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array(TTransVec), np.array(TTransVec))
    DiatData.append(DiatDataTemp)
    
    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)
    

[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2_UMN_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/O2_UMN_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp


In [8]:
def Save_RatesAtT_HDF5( NNRates, PathToHDF5File, TTra, TInt, RatesTypeHDF5, NLevels, Dimension):
    PathToFile    = PathToHDF5File
    HDF5Exist_Flg = path.exists(PathToFile)
    f = h5py.File(PathToFile, 'a')
    
    TStr = Dimension+'/RunI' + str(int(NNRunIdx)) + '/T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'       
    if TStr in f.keys():
        grp       = f[TStr]
        Data      = grp[RatesTypeHDF5]
        Data[...] = NNRates

    else:
        grp       = f.create_group(TStr)
        Proc1     = grp.create_dataset(RatesTypeHDF5, (NLevels[0],NLevels[1]), data=NNRates, compression="gzip", compression_opts=9)

    f.close()
    
def compute_BWDRates(KExcit_NN, TTra):
    Str = 'q_'+str(int(TTra))
    for iLevel in tqdm(range(NLevels[0]), desc='[SurQCT]:     Computing backward rates'):
        for jLevel in range(iLevel+1, NLevels[0]):
#             if (DiatData[1]['EInt'].to_numpy()[jLevel] > DiatData[0]['EInt'].to_numpy()[iLevel]):
            KExcit_NN[iLevel, jLevel] = KExcit_NN[jLevel, iLevel] * (DiatData[1][Str].to_numpy()[jLevel]) / (DiatData[1][Str].to_numpy()[iLevel])

    return KExcit_NN

In [ ]:
try:
    os.makedirs(InputData.FldrtoHDF5NN)
except OSError as e:
    pass

for TTran in TTransVec:
    KExcit_NN_HDF5 = np.zeros((NLevels[0],NLevels[1]))
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.02)
        
    
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] <= DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] > DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] > DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    >= DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    < DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    < DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)  


        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels

        TTranVec              = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
        TTranDataTemp.index   = jIdxVec


        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec

        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec


        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]

        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)

        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            KExcit_NN_FWD          = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact  

        ## BWD Rates
        iiIdxVec              = jIdxVecNo
        jIdxVec               = [iIdx]*jNLevelsNo

        TTranVec              = np.ones((jNLevelsNo))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
        TTranDataTemp.index   = iiIdxVec


        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = iiIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy() 
        jLevelsDataTemp.index = iiIdxVec
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp

        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = iiIdxVec


        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]

        xTempp_BWD            = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)

        if (len(xTempp_BWD[NN_KExcit.xTrainingVar]) > 0):
            KExcit_NN_BWD          = np.exp( NN_KExcit.Model.predict(xTempp_BWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact 
            KExcit_NN_BWD[:,0]     = KExcit_NN_BWD[:,0] / RatioNo

            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_j') for VarName in iLevelsData.columns]

            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_i') for VarName in kLevelsData.columns]

            xTemp_BWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)



        ### Concatenating
            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                xTemp     = pd.concat([xTemp_FWD, xTemp_BWD], axis=0)
                KExcit_NN = np.concatenate((KExcit_NN_FWD, KExcit_NN_BWD), axis=0)
            else:
                xTemp     = xTemp_BWD
                KExcit_NN = KExcit_NN_BWD
        else:
            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
                xTemp     = xTemp_FWD
                KExcit_NN = KExcit_NN_FWD   
        KExcit_NN_HDF5[iIdx,:] = KExcit_NN.reshape((NLevels[1],))
  
    Save_RatesAtT_HDF5(KExcit_NN_HDF5, InputData.PathtoHDF5NN, TTran, TTran, RatesTypeHDF5, NLevels,Dimension) 
    

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 6115/6115 [50:35<00:00,  2.01it/s]
[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:  20%|██        | 1244/6115 [11:29<44:11,  1.84it/s]